In [1]:
# Monitor SC

In [22]:
!pip install matplotlib seaborn pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 34.7 MB/s eta 0:00:00a 0:00:01


In [34]:
import logging
import uuid

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import smartpynector as sp
from utils import *

In [4]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("MonitorSC")

In [5]:
# Constants
READ_URL = "https://graphdb.odissei.nl/repositories/MateuszTest"
WRITE_URL = "https://graphdb.odissei.nl/repositories/MateuszTest/statements"

THERMOSTAT_API_URL = "http://0.0.0.0:8001/thermostat"
PREFIXES = {
    "rdf": "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
    "saref": "https://w3id.org/saref#",
    "xsd": "http://www.w3.org/2001/XMLSchema#",
    "ex": "http://example.org/",
}

MEAS_GRAPH_PATTERN = """?meas rdf:type saref:Measurement .
                        ?meas saref:hasValue ?temp .
                        ?meas saref:isMeasuredIn saref:TemperatureUnit .
                        ?meas saref:hasTimestamp ?timestamp .
                        ?meas saref:isMeasurementOf ?room_id .
                        ?meas saref:relatesToProperty saref:Temperature .
                        ?meas saref:measurementMadeBy ?device_id ."""

H_MEAS_II_GRAPH_PATTERN = """?timeseries rdf:type ex:Timeseries .
                            ?timeseries ex:hasMeasurement ?meas .
                            ?timeseries ex:measuredAfter ?startTimestamp .
                            ?timeseries ex:measuredBefore ?endTimestamp .
                            ?meas rdf:type saref:Measurement .
                            ?meas saref:hasValue ?temp .
                            ?meas saref:hasTimestamp ?timestamp .
                            """

In [6]:
def start_monitor_kb(kb_id, kb_name, kb_description, ke_endpoint):
    register_knowledge_base(kb_id, kb_name, kb_description, ke_endpoint)

    global ask_measurements_ki
    ask_measurements_ki = register_ask_knowledge_interaction(
        H_MEAS_II_GRAPH_PATTERN,
        "ask-measurements",
        kb_id,
        ke_endpoint,
        PREFIXES,
    )

In [7]:
start_monitor_kb(
    "http://example.org/monitor",
    "Monitor",
    "Monitor for storage",
    "http://knowledge_engine:8280/rest/",
)

2023-06-21 23:18:29 INFO registered Monitor
2023-06-21 23:18:29 INFO received issued knowledge interaction id: http://example.org/monitor/interaction/ask-measurements


In [35]:
def plot_temperature(data):
    # Convert data to a DataFrame
    df = pd.DataFrame(data)

    # Convert timestamp and temperature columns to appropriate data types
    df["timestamp"] = df["timestamp"].str.strip('"')
    df["timestamp"] = pd.to_datetime(df["timestamp"])

    df["temp"] = df["temp"].str.strip('"')
    df["temp"] = pd.to_numeric(df["temp"])

    # Create line plot using seaborn
    sns.lineplot(data=df, x="timestamp", y="temp")
    plt.xticks(rotation=90)

    # Show plot
    plt.show()

In [36]:
def graph_data(start_time, end_time):
    global historical_measurements
    historical_measurements = ask(
        [
            {
                "timeseries": f"<{THERMOSTAT_API_URL}/timeseries/{str(uuid.uuid4())}>",
                "startTimestamp": f'"{start_time}"',
                "endTimestamp": f'"{end_time}"',
            }
        ],
        ask_measurements_ki,
        "http://example.org/monitor",
        "http://knowledge_engine:8280/rest/",
    )

    # logger.info(f"Historical measurements: {historical_measurements}")
    if historical_measurements and historical_measurements != [{}]:
        plot_temperature(historical_measurements)
    else:
        logger.info("No historical measurements found.")

In [37]:
from datetime import datetime, timezone

from IPython.display import display
from ipywidgets import Button, DatetimePicker, HBox


def convert_datetimes(start_widget, end_widget):
    # Convert the datetime object from UTC to the local timezone
    start_datetime_obj = (
        start_widget.value.astimezone().replace(microsecond=0).isoformat()
    )
    end_datetime_obj = end_widget.value.astimezone().replace(microsecond=0).isoformat()
    return start_datetime_obj, end_datetime_obj


start_widget = DatetimePicker(description="Start Time:", disabled=False)
start_widget.value = datetime.now(timezone.utc).replace(microsecond=0)

end_widget = DatetimePicker(description="End Time:", disabled=False)
end_widget.value = datetime.now(timezone.utc).replace(microsecond=0)


submit_button = Button(description="Submit", button_style="danger")

display(HBox([start_widget, end_widget, submit_button]))


def on_button_click(button):
    start_timestamp, end_timestamp = convert_datetimes(start_widget, end_widget)
    logger.info(f"Start: {start_timestamp}, End: {end_timestamp}")
    graph_data(start_timestamp, end_timestamp)


submit_button.on_click(on_button_click)